# **Labwork1. Exploring and visualising data with Python**

## **Objective: to gain basic skills in using Python for data research and visualisation.**

## Task:
1. Download the `russia_losses_equipment.csv` file from Kaggle.com  
   [https://www.kaggle.com/datasets/piterfm/2022-ukraine-russian-war](https://www.kaggle.com/datasets/piterfm/2022-ukraine-russian-war)
2. Choose one variant for this laboratory work, using the formula:  
   **N = ord("LLL") % 3 + 1**,  
   where N is the variant number and LLL is the first letter of your name.
3. Tips:  
    a. Use the `csv.reader()` function to read the CSV file.  
    b. Skip the first row (with column headers).  
    c. Do not use loops or other iterative constructs for processing NumPy arrays. Use only slicing and/or universal (vectorized) functions.  
    d. Create functions for tasks 3-7 for each variant.
4. Requirements for plots:  
   a. Set the line type (dashed, etc.);  
   b. The plots should be labeled;  
   c. Add a legend;  
   d. Add a grid and specify the color and line type;  
   e. Add a title to the plot, axis labels, and values on the axes;  
   f. Change the size of the plot (e.g., 8x16 inches) and resolution (e.g., 100 dpi);  
   g. Save the plot to a PNG file.

In [ ]:
N = ord("D") % 3 + 1
print(f"Denys Kolesnychenko's variant: {N}")

## Variant 3. Study of Armored Personnel Carrier (APC) losses.
1. Create a function that takes the filename and column name as parameters and returns a NumPy array with the column values.
2. Using the function from the previous task, create a NumPy array with the values of the "APC" column (Armored Personnel Carrier).
3. Calculate daily APC losses.
4. Find the 5 highest daily APC losses.
5. Determine how many APCs were destroyed in the summer of 2023.
6. Find the average number of destroyed APCs over the 100-500 days of the war.
7. Create a plot of APC losses for the last 200 days (starting from the last date in the dataset).

## Completion of the task

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, date, timedelta
import time
import csv
import os

DATA_FOLDER_PATH = 'data'
LOSSES_FILE_NAME = 'russia_losses_equipment.csv'
LOSSES_FILE_PATH = os.path.join(DATA_FOLDER_PATH, LOSSES_FILE_NAME)
LOSSES_COLUMN_NAME = 'APC'
WAR_START_DATE = date(2022, 2, 24)
# Task 5
START_DATE = date(2023, 6, 1)
END_DATE = date(2023, 8, 31)
# Task 6
START_DATE_NUMBER = 100
END_DATE_NUMBER = 500
# Task 7
LAST_N_DAYS = 200
RESULT_FOLDER_PATH = 'result'
RESULT_FILE_NAME = f'{LOSSES_COLUMN_NAME}_losses_graph_{LAST_N_DAYS}_days.png'
RESULT_FILE_PATH = os.path.join(RESULT_FOLDER_PATH, RESULT_FILE_NAME)


def get_int_column_data_as_numpy_array(source_file, column_name):
    if not os.path.exists(source_file):
        raise FileNotFoundError(f"File '{source_file}' not found.")

    try:
        with open(source_file, 'r', encoding='utf-8') as file:
            reader = csv.reader(file, delimiter=',')
            headers = next(reader)

            if column_name not in headers:
                raise ValueError(f"Column '{column_name}' not found in headers.")

            column_index = headers.index(column_name)

            column_data = []

            for row in reader:
                value = row[column_index]
                if value == "":
                    raise ValueError(f"Missing value in column '{column_name}' at line {reader.line_num}.")
                try:
                    column_data.append(int(value))
                except ValueError:
                    raise ValueError(f"Invalid value '{row[column_index]}' in column '{column_name}' at line {reader.line_num}.")

            column_data.append(0)

            return np.array(column_data)
    except FileNotFoundError:
        raise FileNotFoundError(f"File '{source_file}' cannot be opened.")


try:
    cumulative_losses_data = get_int_column_data_as_numpy_array(LOSSES_FILE_PATH, LOSSES_COLUMN_NAME)
    print(f"Cumulative {LOSSES_COLUMN_NAME} losses for each day of the war:\n{cumulative_losses_data}")
except Exception as e:
    print(f"Error: {e}")


In [ ]:
def calculate_daily_losses(cumulative_losses_data):
    if len(cumulative_losses_data) < 2:
        raise ValueError("Not enough data to calculate daily losses.")

    return -np.diff(cumulative_losses_data)


try:
    daily_losses_data = calculate_daily_losses(cumulative_losses_data)
    print(f"\nDaily {LOSSES_COLUMN_NAME} losses:\n{daily_losses_data}")
except Exception as e:
    print(f"Error: {e}")


In [ ]:
def get_top_n_losses(daily_losses_data, n=5):
    if len(daily_losses_data) < n:
        raise ValueError(f"Not enough values. Requested {n}, available {len(daily_losses_data)}")
    return np.sort(daily_losses_data)[-1:-(n + 1):-1]


def get_top_n_unique_losses(daily_losses_data, n=5):
    unique_losses = np.unique(daily_losses_data)

    if len(unique_losses) < n:
        raise ValueError(f"There are not enough unique values. Requested {n}, available {len(unique_losses)}")

    top_n_unique = get_top_n_losses(unique_losses, n)

    return top_n_unique


try:
    top_5_losses = get_top_n_losses(daily_losses_data)
    print(f"\nTop 5 daily {LOSSES_COLUMN_NAME} losses:\n{top_5_losses}")

    top_5_unique_losses = get_top_n_unique_losses(daily_losses_data)
    print(f"\nTop 5 unique daily {LOSSES_COLUMN_NAME} losses:\n{top_5_unique_losses}")

    top_10_losses = get_top_n_losses(daily_losses_data, 10)
    print(f"\nTop 10 daily {LOSSES_COLUMN_NAME} losses:\n{top_10_losses}")

    top_10_unique_losses = get_top_n_unique_losses(daily_losses_data, 10)
    print(f"\nTop 10 unique daily {LOSSES_COLUMN_NAME} losses:\n{top_10_unique_losses}")
except Exception as e:
    print(f"Error: {e}")


In [ ]:
def validate_period_indices(start_day_number, end_day_number, data_length):
    if start_day_number <= 0:
        raise ValueError("Start day index is out of bounds (should be > 0)")
    if end_day_number >= data_length:
        raise ValueError("End day index is out of bounds (should be < length of data)")
    if start_day_number > end_day_number:
        raise ValueError("Start day index is greater than end day index")


def calculate_war_day(specific_date, war_start_date):
    if isinstance(specific_date, datetime):
        specific_date = specific_date.date()
    elif not isinstance(specific_date, date):
        raise ValueError("specific_date must be a date or datetime object")

    if specific_date < war_start_date:
        raise ValueError("The date can't be earlier than the start of the war")

    days_passed = (specific_date - war_start_date).days + 1
    return days_passed


def calculate_losses_for_period(cumulative_losses_data, start_day_number, end_day_number):
    validate_period_indices(start_day_number, end_day_number, len(cumulative_losses_data))

    start_index = len(cumulative_losses_data) - start_day_number
    end_index = len(cumulative_losses_data) - end_day_number - 1

    start_time = time.perf_counter()
    losses = cumulative_losses_data[end_index] - cumulative_losses_data[start_index]
    end_time = time.perf_counter()

    print(f"Execution time(calculate_losses_for_period): {(end_time - start_time):.6f}")
    return losses


def calculate_losses_sum_for_period(daily_losses_data, start_day_number, end_day_number):
    validate_period_indices(start_day_number, end_day_number, len(daily_losses_data))

    start_index = len(daily_losses_data) - start_day_number + 1
    end_index = len(daily_losses_data) - end_day_number

    start_time = time.perf_counter()
    total_losses = np.sum(daily_losses_data[end_index:start_index])
    end_time = time.perf_counter()

    print(f"Execution time(calculate_losses_sum_for_period): {(end_time - start_time):.6f}")
    return total_losses


try:
    print("\nUsing calculate_losses_for_period:")
    period_losses = calculate_losses_for_period(cumulative_losses_data, 
                                                calculate_war_day(START_DATE, WAR_START_DATE), 
                                                calculate_war_day(END_DATE, WAR_START_DATE))
    print(f"{LOSSES_COLUMN_NAME} losses from {START_DATE} to {END_DATE}: {period_losses}")

    print(
        "\nUsing calculate_losses_sum_for_period:")
    period_losses_sum = calculate_losses_sum_for_period(daily_losses_data, 
                                                        calculate_war_day(START_DATE, WAR_START_DATE), 
                                                        calculate_war_day(END_DATE, WAR_START_DATE))
    print(f"{LOSSES_COLUMN_NAME} losses from {START_DATE} to {END_DATE}: {period_losses_sum}")
except Exception as e:
    print(f"Error: {e}")


In [ ]:
def calculate_average_losses_for_period(cumulative_losses_data, start_day_number, end_day_number):
    validate_period_indices(start_day_number, end_day_number, len(cumulative_losses_data))

    start_index = len(cumulative_losses_data) - start_day_number
    end_index = len(cumulative_losses_data) - end_day_number - 1

    start_time = time.perf_counter()
    losses = cumulative_losses_data[end_index] - cumulative_losses_data[start_index]
    average_losses = losses / (end_day_number - start_day_number + 1)
    end_time = time.perf_counter()

    print(f"Execution time(calculate_average_losses_for_period): {(end_time - start_time):.6f}")
    return average_losses


def calculate_average_losses_sum_for_period(daily_losses_data, start_day_number, end_day_number):
    validate_period_indices(start_day_number, end_day_number, len(daily_losses_data))

    start_index = len(daily_losses_data) - start_day_number + 1
    end_index = len(daily_losses_data) - end_day_number

    start_time = time.perf_counter()
    average_losses = np.mean(daily_losses_data[end_index:start_index])
    end_time = time.perf_counter()

    print(f"Execution time(calculate_average_losses_sum_for_period): {(end_time - start_time):.6f}")
    return average_losses


try:
    print("\nUsing calculate_average_losses_for_period:")
    average_losses = calculate_average_losses_for_period(cumulative_losses_data, START_DATE_NUMBER, END_DATE_NUMBER)
    print(f"Average {LOSSES_COLUMN_NAME} losses from day {START_DATE_NUMBER} to day {END_DATE_NUMBER}: {average_losses:.2f}")

    print("\nUsing calculate_average_losses_sum_for_period:")
    average_losses_sum = calculate_average_losses_sum_for_period(daily_losses_data, START_DATE_NUMBER, END_DATE_NUMBER)
    print(f"Average {LOSSES_COLUMN_NAME} losses from day {START_DATE_NUMBER} to day {END_DATE_NUMBER}: {average_losses_sum:.2f}")
except Exception as e:
    print(f"Error: {e}")


In [ ]:
%matplotlib widget


def get_latest_date(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File '{file_path}' not found.")
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            reader = csv.reader(file)
            headers = next(reader)
            date_index = headers.index('date')
            first_row = next(reader)
            latest_date = datetime.strptime(first_row[date_index], '%Y-%m-%d').date()
            return latest_date
    except FileNotFoundError:
        raise FileNotFoundError(f"File '{file_path}' cannot be opened.")


def prepare_data_for_graph(file_path, cumulative_losses_data, last_n_days):
    latest_date = get_latest_date(file_path)
    start_date = latest_date - timedelta(days=last_n_days - 1)
    dates = [(start_date + timedelta(days=i)) for i in range(last_n_days)]
    last_n_days_cumulative_losses = cumulative_losses_data[last_n_days - 1::-1]
    return dates, last_n_days_cumulative_losses


def plot_cumulative_losses(dates, last_n_days_cumulative_losses, result_file, column_name, last_n_days):
    plt.figure(figsize=(10, 12), dpi=100)
    plt.plot(dates, last_n_days_cumulative_losses, linestyle='--', color='blue', label=f'{column_name} cumulative losses')

    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

    xticks = ax.get_xticks()
    new_ticks = np.array([mdates.date2num(dates[0]), mdates.date2num(dates[-1])])
    combined_xticks = np.concatenate((xticks, new_ticks))
    xticklabels = [tick.get_text() for tick in ax.get_xticklabels()]
    combined_xticklabels = xticklabels + [dates[0].strftime('%Y-%m-%d'), dates[-1].strftime('%Y-%m-%d')]
    ax.set_xticks(combined_xticks)
    ax.set_xticklabels(combined_xticklabels, rotation=45)

    x_range = mdates.date2num(dates[-1]) - mdates.date2num(dates[0])
    x_buffer = x_range * 0.03
    ax.set_xlim(mdates.num2date(mdates.date2num(dates[0]) - x_buffer),
                mdates.num2date(mdates.date2num(dates[-1]) + x_buffer))

    yticks = ax.get_yticks()
    new_ticks = np.array([min(last_n_days_cumulative_losses), max(last_n_days_cumulative_losses)])
    combined_ticks = np.concatenate((yticks, new_ticks))
    ax.set_yticks(combined_ticks)
    ax.set_yticklabels([f'{int(tick)}' for tick in combined_ticks])

    y_buffer = (max(last_n_days_cumulative_losses) - min(last_n_days_cumulative_losses)) * 0.03
    ax.set_ylim(min(last_n_days_cumulative_losses) - y_buffer, max(last_n_days_cumulative_losses) + y_buffer)

    plt.title(f'{column_name} losses in the last {last_n_days} days',
              fontsize=16, fontweight='bold', color='darkred', pad=20)
    plt.xlabel('Date', fontsize=12, labelpad=15, fontweight='bold')
    plt.ylabel('Cumulative losses', fontsize=12, labelpad=15, fontweight='bold')

    plt.grid(True, linestyle='--', color='darkgray', alpha=0.5)

    plt.legend()

    plt.savefig(result_file)
    print(f"\nThe graph is saved to a file: {result_file}")
    plt.show()


try:
    dates, last_n_days_cumulative_losses = prepare_data_for_graph(LOSSES_FILE_PATH, cumulative_losses_data, LAST_N_DAYS)
    plot_cumulative_losses(dates, last_n_days_cumulative_losses, RESULT_FILE_PATH, LOSSES_COLUMN_NAME, LAST_N_DAYS)
except Exception as e:
    print(f"Error: {e}")


In [ ]:
# plt.close()